In [144]:
import pandas as pd
import unicodedata
import re

In [145]:
df = pd.read_csv(
    "../Data Layer/silver/Viagens_2025_LIMPAS.csv",
    sep=";",
    encoding="latin1",
    decimal=",",
    dayfirst=True,
    parse_dates=[
        'Período - Data de início',
        'Período - Data de fim'
    ]
)

In [146]:
df.columns = (
    df.columns
      .str.normalize('NFKD')
      .str.encode('ascii', errors='ignore')
      .str.decode('utf-8')
      .str.lower()
      .str.replace(r'[^\w]+', '_', regex=True)
      .str.strip('_')
)
df = df.rename(columns={
    'periodo_data_de_inicio': 'data_inicio',
    'periodo_data_de_fim': 'data_fim',
    'codigo_do_orgao_superior' : 'codigo_orgao_superior',
    'nome_do_orgao_superior' : 'nome_orgao_superior'
})

In [147]:
dim_tempo = (
    df[['data_inicio']]
    .drop_duplicates()
)

dim_tempo['tempo_id'] = range(1, len(dim_tempo) + 1)
dim_tempo['ano'] = dim_tempo['data_inicio'].dt.year
dim_tempo['mes_numero'] = dim_tempo['data_inicio'].dt.month
dim_tempo['mes_nome'] = dim_tempo['data_inicio'].dt.month_name()

In [148]:
dim_orgao_superior = (
    df[['codigo_orgao_superior', 'nome_orgao_superior']]
    .drop_duplicates()
)

dim_orgao_superior['orgao_superior_id'] = range(1, len(dim_orgao_superior) + 1)


In [149]:
dim_orgao_solicitante = (
    df[['codigo_orgao_solicitante', 'nome_orgao_solicitante', 'codigo_orgao_superior']]
    .drop_duplicates()
)

dim_orgao_solicitante['orgao_solicitante_id'] = range(1, len(dim_orgao_solicitante) + 1)


In [150]:
dim_viajante = (
    df[['cpf_viajante', 'nome', 'cargo', 'descricao_funcao']]
    .drop_duplicates()
)

dim_viajante['viajante_id'] = range(1, len(dim_viajante) + 1)


In [151]:
dim_motivo = (
    df[['motivo']]
    .dropna()
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_motivo['motivo_id'] = dim_motivo.index + 1
dim_motivo = dim_motivo[['motivo_id', 'motivo']]


In [152]:
fato_viagens = (
    df

    # Tempo
    .merge(
        dim_tempo,
        on=['data_inicio'],
        how='left'
    )

    # Órgão superior
    .merge(
        dim_orgao_superior,
        on=['codigo_orgao_superior'],
        how='left'
    )

    # Órgão solicitante
    .merge(
        dim_orgao_solicitante,
        on=['codigo_orgao_solicitante'],
        how='left'
    )

    # Viajante
    .merge(
        dim_viajante,
        on=['cpf_viajante'],
        how='left'
    )

    # Motivo
    .merge(
        dim_motivo,
        on=['motivo'],
        how='left'
    )
)

In [153]:
fato_viagens = fato_viagens[[
    # Chaves estrangeiras
    'tempo_id',
    'orgao_superior_id',
    'orgao_solicitante_id',
    'viajante_id',
    'motivo_id',

    # Métricas
    'valor_diarias',
    'valor_passagens',
    'valor_outros_gastos',
    'valor_devolucao',
    'total_gasto'
]]

In [154]:
fato_viagens = fato_viagens.reset_index(drop=True)
fato_viagens['fato_viagem_id'] = fato_viagens.index + 1

In [155]:
cols = ['fato_viagem_id'] + [c for c in fato_viagens.columns if c != 'fato_viagem_id']
fato_viagens = fato_viagens[cols]

In [156]:
fato_viagens.isna().sum()

fato_viagem_id          0
tempo_id                0
orgao_superior_id       0
orgao_solicitante_id    0
viajante_id             0
motivo_id               0
valor_diarias           0
valor_passagens         0
valor_outros_gastos     0
valor_devolucao         0
total_gasto             0
dtype: int64